In [3]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torch import optim

In [2]:
class rnn(nn.Module):
    
    def __init__(self):
        
        super(rnn, self).__init__()
        
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.fc = nn.Linear(51, 1)
        
    def forward(self, x, future=0):
        outputs = []
        
        # zero initialization
        h_t = torch.zeros(x.size(0), 51, dtype = torch.double)
        c_t = torch.zeros(x.size(0), 51, dtype = torch.double)
        h_t2 = torch.zeros(x.size(0), 51, dtype = torch.double)
        c_t2 = torch.zeros(x.size(0), 51, dtype = torch.double)
        
        for i, h_t in enumerate(x.chunk(x.size(0))):
            h_t, c_t = self.lstm1(x, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            out = self.fc(h_t2)
            
            outputs.append(out)
            
        # for future prediction
        for i in range(future):
            h_t, c_t = self.lstm1(out, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            pred = self.fc(h_t2)
            
            outputs.append(pred)
            
        outputs = torch.stack(outputs, 1).squeeze(2)
        
        return outputs

In [6]:
inputs = [torch.randn(1, 3) for _ in range(5)]

In [7]:
inputs

[tensor([[-1.1696,  0.5191, -0.6949]]),
 tensor([[-0.1844, -0.3195, -0.2793]]),
 tensor([[ 0.4346,  0.2889, -0.5187]]),
 tensor([[0.6234, 1.3628, 1.1370]]),
 tensor([[-0.2190, -0.5818,  1.7597]])]

In [8]:
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))

lstm = nn.LSTM(3, 3)

for i in inputs:
        out, hidden = lstm(i.view(1, 1, -1), hidden)

In [9]:
out

tensor([[[-0.0563,  0.2581,  0.0826]]], grad_fn=<StackBackward>)